# Stabilized Collective Loan in Marlowe

## Sequence Diagram

## Preliminaries

### Versions

Make a note of the versions of the Marlowe repository and software.

In [1]:
marlowe-cli --version

marlowe-cli 0.0.6.0


In [2]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [3]:
git rev-parse HEAD

76ef66b89bf8e24105dbf22f5e68335b7d5f649e


### Select the Cardano network

Use the `preview` networks, which is in the Babbage era.

In [4]:
export CARDANO_TESTNET_MAGIC=2
export CARDANO_NODE_SOCKET_PATH=node.socket

### Current Time

We'll start the contract at the current time.

In [5]:
NOW=$(($(date -u +%s) * 1000))
echo $NOW

1661915651000


Compute time constants, for convenience when setting time intervals for transactions.

In [6]:
MINUTE=$((60 * 1000))
HOUR=$((60 * MINUTE))
DAY=$((24 * HOUR))

There is a minimum Ada requirement associated with a native token.

In [7]:
ADA=1000000
MIN_ADA=$((2 * ADA))
echo $MIN_ADA

2000000


### Contract Parameters

Set parameters for the contract. This contract scales to large numbers of lenders and borrowers, but that will involve many transactions and much time if run on-chain.

In [8]:
BASENAME=stabilized-collective-loan

DJED_POLICY=76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf
DJED_NAME=DjedUSD

N_STABILIZERS=1
N_INVESTORS=3
N_BORROWERS=5

N_STEPS=20

MINIMUM_STABILIZATION=1000000
MAXIMUM_STABILIZATION=10000000

MINIMUM_INVESTMENT=500000
MAXIMUM_INVESTMENT=5000000

MINIMUM_LOAN=100000
MAXIMUM_LOAN=1000000

PERCENT_STABILIZATION=40
PERCENT_INTEREST=10

TIMEOUT_START=$((NOW + 1 * HOUR))
TIMEOUT_DELTA=$((1 * HOUR))
TIMEOUT_FINAL=$((NOW + 1000 * DAY))

ADMINISTRATOR_ROLE=A
STABILIZER_PREFIX=S
INVESTOR_PREFIX=I
BORROWER_PREFIX=B

### Helper functions for roles

In [9]:
function stabilizer() {
  echo "$STABILIZER_PREFIX$1"
}

In [10]:
function investor() {
  echo "$INVESTOR_PREFIX$1"
}

In [11]:
function borrower() {
  echo "$BORROWER_PREFIX$1"
}

### Helper functions for state

In [12]:
function show_state() {
  jq .tx.state $BASENAME-$1.marlowe | json2yaml
}

In [13]:
function show_choices() {
  jq .tx.state.choices $BASENAME-$1.marlowe | json2yaml
}

In [14]:
function show_nonchoices() {
  jq '.tx.state | {accounts : .accounts, boundValues : .boundValues}' $BASENAME-$1.marlowe | json2yaml
}

### Mint the Djed

The contract will use fake Djed for payments.

In [15]:
DJED_NAME=DjedUSD

In [16]:
DJED_POLICY=76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf

In [17]:
DJED=$DJED_POLICY.$DJED_NAME
echo $DJED

76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.DjedUSD


In `cardano-cli` the token name is printed in hexadecimal, so we have . . .

In [18]:
echo $DJED_POLICY.$(echo -n $DJED_NAME | basenc --base16 | tr '[:upper:]' '[:lower:]')

76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.446a6564555344


### Mint the Role Tokens

There is one role token for the administrator.

Mint the role tokens.

In [19]:
ROLES_POLICY=9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d605064e78

## Generate the Contract

Run a Haskell program that generates the contract.

In [20]:
runhaskell << EOI

{-# LANGUAGE NumericUnderscores #-}
{-# LANGUAGE OverloadedStrings  #-}
{-# LANGUAGE RecordWildCards    #-}


module Main (
  main
) where


import Control.Monad.Writer (Writer, runWriter, tell)
import Data.Aeson ((.=), encodeFile,object)
import Data.Default (Default(..))
import Data.Foldable (foldrM)
import Data.Maybe (fromMaybe)
import Data.String (fromString)
import Language.Marlowe.Core.V1.Semantics.Types
import Plutus.V1.Ledger.Api (BuiltinByteString, POSIXTime(..), TokenName)
import Language.Marlowe.CLI.Merkle (deepMerkleize)
import Language.Marlowe.CLI.Types (Continuations)


data Scenario =
  Scenario
  {
    nSteps               :: Int
  , timeoutStart         :: Integer
  , timeoutDelta         :: Integer
  , timeoutFinal         :: POSIXTime
  , nStabilizers         :: Int
  , nInvestors           :: Int
  , nBorrowers           :: Int
  , administratorName    :: String
  , stabilizerPrefix     :: String
  , investorPrefix       :: String
  , borrowerPrefix       :: String
  , stabilizeBound       :: [Bound]
  , investBound          :: [Bound]
  , loanBound            :: [Bound]
  , percentInterest      :: Integer
  , percentStabilization :: Integer
  , djed                 :: Token
  }

instance Default Scenario where
  def = 
    Scenario
    {
      nSteps               = $N_STEPS
    , timeoutStart         = $TIMEOUT_START
    , timeoutDelta         = $TIMEOUT_DELTA
    , timeoutFinal         = POSIXTime $TIMEOUT_FINAL
    , nStabilizers         = $N_STABILIZERS
    , nInvestors           = $N_INVESTORS
    , nBorrowers           = $N_BORROWERS
    , administratorName    = "$ADMINISTRATOR_ROLE"
    , stabilizerPrefix     = "$STABILIZER_PREFIX"
    , investorPrefix       = "$INVESTOR_PREFIX"
    , borrowerPrefix       = "$BORROWER_PREFIX"
    , stabilizeBound       = [Bound $MINIMUM_STABILIZATION $MAXIMUM_STABILIZATION]
    , investBound          = [Bound $MINIMUM_INVESTMENT    $MAXIMUM_INVESTMENT   ]
    , loanBound            = [Bound $MINIMUM_LOAN          $MAXIMUM_LOAN         ]
    , percentInterest      = $PERCENT_INTEREST
    , percentStabilization = $PERCENT_STABILIZATION
    , djed                 = Token "$DJED_POLICY" "$DJED_NAME"
    }


main :: IO ()
main =
  let
    (contract, continuations) = buildContract def
  in
    encodeFile "$BASENAME.merkleization"
      $ object
        [
          "contract"      .= contract
        , "continuations" .= continuations
        ]


buildContract :: Scenario -> (Contract, Continuations)
buildContract scenario@Scenario{..} =
  let
    timeouts = [POSIXTime $ timeoutStart + toInteger i * timeoutDelta | i <- [1..nSteps]]
  in
    runWriter $ foldrM (makeStep scenario) Close timeouts


defaultBound :: [Bound]
defaultBound = [Bound 1 1_000_000_000_000]


makeParty :: String -> Int -> Party
makeParty name i = Role . fromString $ name <> show i


deposit :: Scenario
        -> String
        -> String
        -> Int
        -> Maybe Party
        -> [Bound]
        -> (Value Observation -> Contract -> Contract)
        -> Contract
        -> [Case Contract]
deposit Scenario{..} name prefix i recipient bound f continuation =
  let
    party = makeParty prefix i
    party' = fromMaybe party recipient
    choice = ChoiceId (fromString name) party
    value = ChoiceValue choice
  in
    [
      Case (Choice choice bound)
        continuation
    , Case (Deposit party' party djed value)
        $ f value continuation
    ]


stabilization :: ValueId
stabilization = ValueId "Stabilization"


stabilize :: Scenario -> Int -> Contract -> [Case Contract]
stabilize scenario@Scenario{..} i =
  deposit scenario "Stabilize" stabilizerPrefix i Nothing stabilizeBound
    (stabilization =+)
 

assets :: ValueId
assets = ValueId "Assets"


equity :: Int -> ValueId
equity = ValueId . ("Equity " <>) . fromString . show


invest :: Scenario -> Int -> Contract -> [Case Contract]
invest scenario@Scenario{..} i =
  deposit scenario "Invest" investorPrefix i Nothing investBound
    $ \value -> (assets   =+ value)
              . (equity i =+ value)


withdraw :: Scenario
         -> Int
         -> Contract
         -> [Case Contract]
withdraw Scenario{..} i continuation =
  let
    party = makeParty investorPrefix i
    choice = ChoiceId "Withdraw" party
    value = ChoiceValue choice
    funds = AvailableMoney party djed
  in
    [
      Case (Choice choice defaultBound)
        $ If (value @<= funds)
            (
              Pay party (Party party) djed value
                $ assets   =- value
                $ equity i =- value
                $ continuation
            )
            (
              Pay party (Party party) djed funds
                $ assets   =- funds
                $ equity i =- funds
                $ continuation
            )
    ]


lend :: Scenario
     -> Party
     -> Value Observation
     -> Value Observation
     -> Int
     -> Contract
     -> Contract
lend Scenario{..} borrower principal interest i =
  let
    party = makeParty investorPrefix i
    funds = AvailableMoney party djed
    assets' = UseValue assets
    principal' = (principal @* funds) @/ assets'
    interest'  = (interest  @* funds) @/ assets'
  in
    Pay party (Party borrower) djed principal'
      . (equity i =+ interest')


liabilities :: ValueId
liabilities = ValueId "Liabilities"

loan :: Scenario
     -> Int
     -> Contract
     -> [Case Contract]
loan scenario@Scenario{..} i continuation =
  let
    party = Role $ fromString administratorName
    borrower = makeParty borrowerPrefix i
    choice = ChoiceId "Loan" party
    principal = ChoiceValue choice
    assets' = UseValue assets
    stabilization' = UseValue stabilization
  in
    [
      Case (Choice choice loanBound)
        $ let
            interest = principal @% percentInterest
            debt = principal @+ interest
          in
            If ((principal @<= assets') @&& (stabilization' @>= (debt @% percentStabilization)))
              (
                flip (foldr $ lend scenario borrower principal interest) [1..nInvestors]
                  $ liabilities =+ debt
                  $ assets      =- principal
                  $ continuation
              )
              continuation
    ]


restore :: Scenario
        -> Value Observation
        -> Int
        -> Contract
        -> Contract
restore Scenario{..} repayment i =
  let
    administrator = Role $ fromString administratorName
    party = makeParty investorPrefix i
    equity' = UseValue $ equity i
    assets' = AvailableMoney party djed
    liabilities' = UseValue liabilities
    share = (repayment @* (equity' @- assets')) @/ liabilities'
  in
    Pay administrator (Account party) djed share


repay :: Scenario
      -> Int
      -> Contract
      -> [Case Contract]
repay scenario@Scenario{..} i =
  deposit scenario "Repay" borrowerPrefix i (Just . Role $ fromString administratorName) defaultBound
    $ \repayment -> flip (foldr $ restore scenario repayment) [1..nInvestors]
                      . (liabilities =- repayment)
                      . (assets      =+ repayment)


gather :: Scenario
       -> Value Observation
       -> ValueId
       -> String
       -> Int
       -> Contract
       -> Contract
gather Scenario{..} amount basis prefix i = 
  let
    administrator = Role $ fromString administratorName
    party = makeParty prefix i
    funds = AvailableMoney party djed
    basis' = UseValue basis
    share = (amount @* funds) @/ basis'
  in
    Pay party (Account administrator) djed share


scatter :: Scenario
        -> Value Observation
        -> ValueId
        -> String
        -> Int
        -> Contract
        -> Contract
scatter Scenario{..} amount basis prefix i =
  let
    administrator = Role $ fromString administratorName
    party = makeParty prefix i
    funds = AvailableMoney party djed
    basis' = UseValue basis
    share = (amount @* funds) @/ basis'
  in
    Pay administrator (Account party) djed share


terminate :: Scenario
          -> [Case Contract]
terminate scenario@Scenario{..} =
  let
    party = Role $ fromString administratorName
    choice = ChoiceId "Terminate" party
    liabilities' = UseValue liabilities
  in
    [
      Case (Choice choice [Bound 1 1])
        $ If (liabilities' @>= Constant 0)
          (
              flip (foldr $ gather  scenario liabilities' stabilization stabilizerPrefix) [1..nStabilizers]
            $ flip (foldr $ scatter scenario liabilities' assets        investorPrefix  ) [1..nInvestors  ]
              Close
          )
          (
              flip (foldr $ gather  scenario (NegValue liabilities') assets        investorPrefix  ) [1..nInvestors  ]
            $ flip (foldr $ scatter scenario (NegValue liabilities') stabilization stabilizerPrefix) [1..nStabilizers]
              Close
          )
    ]


makeStep :: Scenario
         -> Timeout
         -> Contract
         -> Writer Continuations Contract
makeStep scenario@Scenario{..} timeout continuation =
  let
  in
    deepMerkleize
      . When
        (
          concat
            $  [stabilize scenario i continuation | i <- [1..nStabilizers]]
            <> [invest    scenario i continuation | i <- [1..nInvestors]  ]
            <> [withdraw  scenario i continuation | i <- [1..nInvestors]  ]
            <> [loan      scenario i continuation | i <- [1..nBorrowers]  ]
            <> [repay     scenario i continuation | i <- [1..nBorrowers]  ]
            <> [terminate scenario                                        ]
        )
      timeout
    =<< merkleizeTimeout timeoutFinal continuation


merkleizeTimeout :: Timeout
                 -> Contract
                 -> Writer Continuations Contract
merkleizeTimeout timeout continuation =
  deepMerkleize
    $ When
      [
        Case (Notify TrueObs )
        continuation
      ]
      timeout
      Close


(=+) :: ValueId -> Value Observation -> (Contract -> Contract)
variable =+ value = Let variable $ UseValue variable @+ value

(=-) :: ValueId -> Value Observation -> (Contract -> Contract)
variable =- value = Let variable $ UseValue variable @- value

(=:) :: ValueId -> Integer -> (Contract -> Contract)
variable =: value = Let variable $ Constant value

(@%) :: Value Observation -> Integer -> Value Observation
value @% rate = DivValue (MulValue value $ Constant rate) (Constant 100)

(@+%) :: Value Observation -> Integer -> Value Observation
value @+% rate = DivValue (MulValue value (AddValue (Constant rate) $ Constant 0)) (Constant 100)

(@+) :: Value Observation -> Value Observation -> Value Observation
x @+ y = AddValue x y

(@-) :: Value Observation -> Value Observation -> Value Observation
x @- y = SubValue x y

(@*) :: Value Observation -> Value Observation -> Value Observation
x @* y = MulValue x y

(@/) :: Value Observation -> Value Observation -> Value Observation
x @/ y = DivValue x y

(@<) :: Value Observation -> Value Observation -> Observation
x @< y = ValueLT x y

(@<=) :: Value Observation -> Value Observation -> Observation
x @<= y = ValueLE x y

(@>) :: Value Observation -> Value Observation -> Observation
x @> y = ValueGT x y

(@>=) :: Value Observation -> Value Observation -> Observation
x @>= y = ValueGE x y

(@==) :: Value Observation -> Value Observation -> Observation
x @== y = ValueEQ x y

(@&&) :: Observation -> Observation -> Observation
x @&& y = AndObs x y

(@||) :: Observation -> Observation -> Observation
x @|| y = OrObs x y

EOI

How large is the contract?

In [21]:
ls -lh $BASENAME.merkleization

-rw-rw-r-- 1 bbush bbush-upg 4.2M Aug 30 21:14 stabilized-collective-loan.merkleization


How many `Case` statements does it contain?

In [22]:
sed -e 's/case/&\n/g' $BASENAME.merkleization | grep case | wc -l

13860


How many merkleizations does it contain?

In [23]:
jq '.continuations | length' $BASENAME.merkleization

360


Separate the contract from the merkleizations.

In [24]:
jq .contract $BASENAME.merkleization > $BASENAME-1.contract

## Set the initial state

The Seller will deposit 2 Ada when the contract is created.

In [25]:
yaml2json << EOI > $BASENAME-1.state
accounts:
- - - role_token: $ADMINISTRATOR_ROLE
    - currency_symbol: ''
      token_name: ''
  - $MIN_ADA
boundValues: []
choices: []
minTime: $NOW
EOI

In [26]:
cat $BASENAME-1.state

{"accounts":[[[{"role_token":"A"},{"currency_symbol":"","token_name":""}],2000000]],"boundValues":[],"choices":[],"minTime":1661915651000}


## Simulate and Execute the Contract

### 1. Initialize the contract information

Create a `.marlowe` file that contains all of the information needed to start the contract. Note that we merkleization the contract because of its large size.

In [27]:
n=1

In [28]:
marlowe-cli run initialize --roles-currency $ROLES_POLICY        \
                           --contract-file $BASENAME-$n.contract \
                           --state-file    $BASENAME-$n.state    \
                           --out-file      $BASENAME-$n.partial  \
                           --print-stats


Validator size: 12412
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 18768100, exBudgetMemory = ExMemory 81700}


Manually merge the continuations into the `.marlowe` file.

In [29]:
jq -s '.[0] * {tx : .[1]}' $BASENAME-1.partial $BASENAME.merkleization > $BASENAME-1.marlowe

The minimum Ada deposit has been recorded.

In [30]:
show_state $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
boundValues: []
choices: []
minTime: 1661915651000


In [31]:
DEADLINE=$((NOW + 59 * MINUTE))

### 2. Stabilzer 1 deposits 5M Djed

In [32]:
STABILIZER_DEPOSIT=5000000

In [33]:
n=$((n+1))

In [34]:
marlowe-cli run prepare --choice-party Role=$(stabilizer 1)            \
                        --choice-name Stabilize                        \
                        --choice-number $STABILIZER_DEPOSIT            \
                        --invalid-before $NOW                          \
                        --invalid-hereafter $((DEADLINE - 1 * MINUTE)) \
                        --marlowe-file $BASENAME-$((n-1)).marlowe      \
                        --out-file     $BASENAME-$n.marlowe            \
                        --print-stats


Datum size: 2581


In [35]:
show_choices $n

- - choice_name: Stabilize
    choice_owner:
      role_token: S1
  - 5000000


In [36]:
n=$((n+1))

In [37]:
marlowe-cli run prepare --deposit-account Role=$(stabilizer 1)         \
                        --deposit-party Role=$(stabilizer 1)           \
                        --deposit-token $DJED                          \
                        --deposit-amount $STABILIZER_DEPOSIT           \
                        --invalid-before $NOW                          \
                        --invalid-hereafter $((DEADLINE - 1 * MINUTE)) \
                        --marlowe-file $BASENAME-$((n-1)).marlowe      \
                        --out-file     $BASENAME-$n.marlowe            \
                        --print-stats


Datum size: 2662


In [38]:
show_nonchoices $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: S1
    - currency_symbol: 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf
      token_name: DjedUSD
  - 5000000
boundValues:
- - Stabilization
  - 5000000


### 3. Investor 1 deposits 1M Djed

In [39]:
INVESTOR_DEPOSIT=1000000

In [40]:
n=$((n+1))

In [41]:
marlowe-cli run prepare --choice-party Role=$(investor 1)              \
                        --choice-name Invest                           \
                        --choice-number $INVESTOR_DEPOSIT              \
                        --invalid-before $NOW                          \
                        --invalid-hereafter $((DEADLINE - 1 * MINUTE)) \
                        --marlowe-file $BASENAME-$((n-1)).marlowe      \
                        --out-file     $BASENAME-$n.marlowe            \
                        --print-stats


Datum size: 2685


In [42]:
show_choices $n

- - choice_name: Stabilize
    choice_owner:
      role_token: S1
  - 5000000
- - choice_name: Invest
    choice_owner:
      role_token: I1
  - 1000000


In [43]:
n=$((n+1))

In [44]:
marlowe-cli run prepare --deposit-account Role=$(investor 1)           \
                        --deposit-party Role=$(investor 1)             \
                        --deposit-token $DJED                          \
                        --deposit-amount $INVESTOR_DEPOSIT             \
                        --invalid-before $NOW                          \
                        --invalid-hereafter $((DEADLINE - 1 * MINUTE)) \
                        --marlowe-file $BASENAME-$((n-1)).marlowe      \
                        --out-file     $BASENAME-$n.marlowe            \
                        --print-stats


Datum size: 2777


In [45]:
show_nonchoices $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: S1
    - currency_symbol: 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf
      token_name: DjedUSD
  - 5000000
- - - role_token: I1
    - currency_symbol: 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf
      token_name: DjedUSD
  - 1000000
boundValues:
- - Stabilization
  - 5000000
- - Assets
  - 1000000
- - Equity 1
  - 1000000


### 4. Investor 2 deposits 500k Djed

In [46]:
INVESTOR_DEPOSIT=500000

In [47]:
n=$((n+1))

In [48]:
marlowe-cli run prepare --choice-party Role=$(investor 2)              \
                        --choice-name Invest                           \
                        --choice-number $INVESTOR_DEPOSIT              \
                        --invalid-before $NOW                          \
                        --invalid-hereafter $((DEADLINE - 1 * MINUTE)) \
                        --marlowe-file $BASENAME-$((n-1)).marlowe      \
                        --out-file     $BASENAME-$n.marlowe            \
                        --print-stats


Datum size: 2800


In [49]:
show_choices $n

- - choice_name: Stabilize
    choice_owner:
      role_token: S1
  - 5000000
- - choice_name: Invest
    choice_owner:
      role_token: I1
  - 1000000
- - choice_name: Invest
    choice_owner:
      role_token: I2
  - 500000


In [50]:
n=$((n+1))

In [51]:
marlowe-cli run prepare --deposit-account Role=$(investor 2)           \
                        --deposit-party Role=$(investor 2)             \
                        --deposit-token $DJED                          \
                        --deposit-amount $INVESTOR_DEPOSIT             \
                        --invalid-before $NOW                          \
                        --invalid-hereafter $((DEADLINE - 1 * MINUTE)) \
                        --marlowe-file $BASENAME-$((n-1)).marlowe      \
                        --out-file     $BASENAME-$n.marlowe            \
                        --print-stats


Warnings:
  TransactionShadowing "Assets" 1000000 1500000
Datum size: 2876


In [52]:
show_nonchoices $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: S1
    - currency_symbol: 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf
      token_name: DjedUSD
  - 5000000
- - - role_token: I1
    - currency_symbol: 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf
      token_name: DjedUSD
  - 1000000
- - - role_token: I2
    - currency_symbol: 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf
      token_name: DjedUSD
  - 500000
boundValues:
- - Stabilization
  - 5000000
- - Assets
  - 1500000
- - Equity 1
  - 1000000
- - Equity 2
  - 500000


### 5. Administrator lends 900k Djed to Borrower 1

In [53]:
LOAN_AMOUNT=900000

In [54]:
n=$((n+1))

In [55]:
marlowe-cli run prepare --choice-party Role=$ADMINISTRATOR_ROLE        \
                        --choice-name Loan                             \
                        --choice-number $LOAN_AMOUNT                   \
                        --invalid-before $NOW                          \
                        --invalid-hereafter $((DEADLINE - 1 * MINUTE)) \
                        --marlowe-file $BASENAME-$((n-1)).marlowe      \
                        --out-file     $BASENAME-$n.marlowe            \
                        --print-stats


Warnings:
  TransactionShadowing "Equity 1" 1000000 1024000
  TransactionShadowing "Equity 2" 500000 512000
  TransactionNonPositivePay "I3" (Party "B1") (Token "76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf" "DjedUSD") 0
  TransactionShadowing "Assets" 1500000 600000
Datum size: 2931
Payment 1
  Acccount: "I1"
  Payee: Party "B1"
  Ada: 0.000000
  76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf."DjedUSD": 600000
Payment 2
  Acccount: "I2"
  Payee: Party "B1"
  Ada: 0.000000
  76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf."DjedUSD": 300000


In [56]:
show_nonchoices $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: S1
    - currency_symbol: 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf
      token_name: DjedUSD
  - 5000000
- - - role_token: I1
    - currency_symbol: 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf
      token_name: DjedUSD
  - 400000
- - - role_token: I2
    - currency_symbol: 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf
      token_name: DjedUSD
  - 200000
boundValues:
- - Stabilization
  - 5000000
- - Assets
  - 600000
- - Equity 1
  - 1024000
- - Equity 2
  - 512000
- - Equity 3
  - 0
- - Liabilities
  - 990000


In [57]:
echo $((1024000 + 512000 - 990000))

546000


### 6. Borrower only pays back 500k of the 990k Djed owed

In [58]:
BORROWER_REPAYMENT=500000

In [59]:
n=$((n+1))

In [60]:
marlowe-cli run prepare --choice-party Role=$(borrower 1)              \
                        --choice-name Repay                            \
                        --choice-number $BORROWER_REPAYMENT            \
                        --invalid-before $NOW                          \
                        --invalid-hereafter $((DEADLINE - 1 * MINUTE)) \
                        --marlowe-file $BASENAME-$((n-1)).marlowe      \
                        --out-file     $BASENAME-$n.marlowe            \
                        --print-stats


Datum size: 2953


In [61]:
show_choices $n

- - choice_name: Stabilize
    choice_owner:
      role_token: S1
  - 5000000
- - choice_name: Invest
    choice_owner:
      role_token: I1
  - 1000000
- - choice_name: Invest
    choice_owner:
      role_token: I2
  - 500000
- - choice_name: Loan
    choice_owner:
      role_token: A
  - 900000
- - choice_name: Repay
    choice_owner:
      role_token: B1
  - 500000


In [62]:
n=$((n+1))

In [63]:
marlowe-cli run prepare --deposit-account Role=$ADMINISTRATOR_ROLE     \
                        --deposit-party Role=$(borrower 1)             \
                        --deposit-token $DJED                          \
                        --deposit-amount $BORROWER_REPAYMENT           \
                        --invalid-before $NOW                          \
                        --invalid-hereafter $((DEADLINE - 1 * MINUTE)) \
                        --marlowe-file $BASENAME-$((n-1)).marlowe      \
                        --out-file     $BASENAME-$n.marlowe            \
                        --print-stats


Warnings:
  TransactionNonPositivePay "A" (Account "I3") (Token "76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf" "DjedUSD") 0
  TransactionShadowing "Liabilities" 990000 490000
  TransactionShadowing "Assets" 600000 1100000
Datum size: 3008
Payment 1
  Acccount: "A"
  Payee: Account "I1"
  Ada: 0.000000
  76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf."DjedUSD": 315151
Payment 2
  Acccount: "A"
  Payee: Account "I2"
  Ada: 0.000000
  76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf."DjedUSD": 157575


In [64]:
show_nonchoices $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: S1
    - currency_symbol: 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf
      token_name: DjedUSD
  - 5000000
- - - role_token: I1
    - currency_symbol: 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf
      token_name: DjedUSD
  - 715151
- - - role_token: I2
    - currency_symbol: 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf
      token_name: DjedUSD
  - 357575
- - - role_token: A
    - currency_symbol: 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf
      token_name: DjedUSD
  - 27274
boundValues:
- - Stabilization
  - 5000000
- - Assets
  - 1100000
- - Equity 1
  - 1024000
- - Equity 2
  - 512000
- - Equity 3
  - 0
- - Liabilities
  - 490000


### 7. Administrator closes the pool

In [65]:
n=$((n+1))

In [66]:
marlowe-cli run prepare --choice-party Role=$ADMINISTRATOR_ROLE        \
                        --choice-name Terminate                        \
                        --choice-number 1                              \
                        --invalid-before $NOW                          \
                        --invalid-hereafter $((DEADLINE - 1 * MINUTE)) \
                        --marlowe-file $BASENAME-$((n-1)).marlowe      \
                        --out-file     $BASENAME-$n.marlowe            \
                        --print-stats


Warnings:
  TransactionNonPositivePay "A" (Account "I3") (Token "76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf" "DjedUSD") 0
Datum size: 270
Payment 1
  Acccount: "S1"
  Payee: Account "A"
  Ada: 0.000000
  76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf."DjedUSD": 490000
Payment 2
  Acccount: "A"
  Payee: Account "I1"
  Ada: 0.000000
  76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf."DjedUSD": 318567
Payment 3
  Acccount: "A"
  Payee: Account "I2"
  Ada: 0.000000
  76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf."DjedUSD": 159283
Payment 4
  Acccount: "A"
  Payee: Party "A"
  Ada: 2.000000
Payment 5
  Acccount: "S1"
  Payee: Party "S1"
  Ada: 0.000000
  76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf."DjedUSD": 4510000
Payment 6
  Acccount: "I1"
  Payee: Party "I1"
  Ada: 0.000000
  76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf."DjedUSD": 1033718
Payment 7
  Acccount: "I2"
  Payee: Party "I2"
  Ada: 0.000000
  76ea6a9aad89b094

In [67]:
show_nonchoices $n

accounts: []
boundValues:
- - Stabilization
  - 5000000
- - Assets
  - 1100000
- - Equity 1
  - 1024000
- - Equity 2
  - 512000
- - Equity 3
  - 0
- - Liabilities
  - 490000
